In [9]:
def test_model(model, real_data_test, missing_data_test, mask_test, loss_function, step_per_epoch):
    model.eval()
    test_loss = []
    # batch_size = real_data_test.shape[0]
    # print(batch_size)
    real_dataset = torch.empty((0, lag_size, input_size), dtype=torch.float32).to(device)
    gen_dataset = torch.empty((0, lag_size, input_size), dtype=torch.float32).to(device)
    mask = torch.empty((0, lag_size, input_size), dtype=torch.float32).to(device)
    with torch.no_grad():
        count = 0
        for step in range(step_per_epoch):
            real_data, real_label = gen_real_batch(batch_size, count, real_data_test)

            z_input, mask_input = gen_z_input(batch_size, count, missing_data_test, mask_test)
            
            imputed_results = model(z_input)
            L2 = torch.sum((real_data - imputed_results)**2)
            loss = loss_function(imputed_results, real_data) + L2
            test_loss.append(loss.item())

            real_dataset = torch.cat([real_dataset, real_data], dim=0)
            gen_dataset = torch.cat([gen_dataset, imputed_results], dim=0)
            mask = torch.cat([mask, mask_input], dim=0)

            count += 1

    return real_dataset, gen_dataset, test_loss, mask
